# Make geojson for pulling MODIS data
- Read the site metadata generated in the previous step
- Calculate a 4km x 4km bounding box in WGS84
- Ensure time format is compatible with colab script

In [ ]:
import pandas as pd
from shapely.geometry import Polygon
from pyproj import Proj, transform, CRS
from pyproj.aoi import AreaOfInterest
from pyproj.database import query_utm_crs_info
import geopandas as gpd
from pathlib import Path

In [ ]:
META_DIR = Path('data/meta')
meta = pd.read_csv(META_DIR / 'processed_site_meta.csv')
meta

In [ ]:
def get_4km_polygon_wkt(lat, lon):
    utm_crs_info = query_utm_crs_info(
        area_of_interest=AreaOfInterest(west_lon_degree=lon, south_lat_degree=lat, east_lon_degree=lon, north_lat_degree=lat),
        datum_name="WGS 84"
    )[0]
    utm_crs = CRS.from_epsg(utm_crs_info.code)

    # Projected coordinate system for accurate distance measurement
    proj_utm = Proj(utm_crs)

    # Convert the given lat/lon in WGS84 to UTM coordinates
    x, y = proj_utm(lon, lat)

    # Define the offsets (2km in each direction to form a 4km box)
    offset = 2000  # meters

    # Calculate the corners of the box in UTM coordinates
    bottom_left = (x - offset, y - offset)
    bottom_right = (x + offset, y - offset)
    top_right = (x + offset, y + offset)
    top_left = (x - offset, y + offset)

    # Create a polygon from these corners
    box = Polygon([bottom_left, bottom_right, top_right, top_left, bottom_left])

    # Optionally, convert the polygon back to geographic coordinates (WGS84)
    proj_wgs84 = Proj(proj='latlong', datum='WGS84')
    lon_lat_polygon = transform(proj_utm, proj_wgs84, *box.exterior.xy)

    # Create a new polygon in geographic coordinates
    geo_polygon = Polygon(zip(lon_lat_polygon[0], lon_lat_polygon[1]))

    return geo_polygon

meta['geometry'] = meta.apply(lambda row: get_4km_polygon_wkt(row['LOCATION_LAT'], row['LOCATION_LON']), axis=1)
meta

In [ ]:
gdf = gpd.GeoDataFrame(meta, crs='EPSG:4326', geometry=meta['geometry'])
gdf.to_file(META_DIR / 'sites.geojson', driver='GeoJSON')